# ACTIVIDAD DE CLASIFICACIÓN DE TEXTO

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python". 

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

# Librerías

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Descarga de Datos

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

In [3]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
print(data_dir)

C:\Users\Del Cerro Recuero\.keras\datasets\20_newsgroup


In [5]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['37261', '37913', '37914', '37915', '37916']


In [6]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "39625").read())

Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!howland.reston.ans.net!newsserver.jvnc.net!castor.hahnemann.edu!hal.hahnemann.edu!brennan
From: brennan@hal.hahnemann.edu
Newsgroups: comp.graphics
Subject: .GIFs on a Tek401x ??
Date: 15 MAY 93 14:29:54 EST
Organization: Hahnemann University
Lines: 14
Message-ID: <15MAY93.14295461@hal.hahnemann.edu>
NNTP-Posting-Host: hal.hahnemann.edu


      I was skimming through a few gophers and bumped into one at NIH
   with a database that included images in .GIF format.  While I have
   not yet worked out the kinks of getting the gopher client to call
   an X viewer, I figure that the majority of the users here are not
   in an X11 environment - instead using DOS and MS-Kermit.

      With Kermit supporting Tek4010 emulation for graphics display,
   does anyone know of a package that would allow a Tek to display a
   .GIF image?  It would be of more use to the local population to
   plug something of this sort in as the 'picture' command in

In [7]:
#Algunos archivos de la categoria "talk.politics.misc"
fnames = os.listdir(data_dir / "talk.politics.misc")
print("Number of files in talk.politics.misc:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['124146', '176845', '176846', '176847', '176849']


In [8]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

In [9]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found


KeyboardInterrupt: 

# Mezclando los datos para separarlos en Traning y Test

In [ ]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

# Tokenización de las palabras con TextVectorization 

In [ ]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
vectorizer.get_vocabulary()[:5]

In [ ]:
len(vectorizer.get_vocabulary())

# Viendo la salida de Vectorizer

In [ ]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

In [ ]:
output

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

# Tokenización de los datos de entrenamiento y validación

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Creación y entrenamiento del modelo

In [ ]:
# pon aquí tu código
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import Adam

# Creación del modelo
model = Sequential()
model.add(Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=64, input_length=200))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(class_names), activation='softmax'))

# Compilación del modelo
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento del modelo
history = model.fit(x_train, y_train,
                    epochs=30,
                    batch_size=128,
                    validation_data=(x_val, y_val))

print(model.summary())


### 1.	Utilizando el tokenizador de spacy, que ya conoces, calcula el número promedio de tokens de una muestra de 15 ficheros de la categoría ‘com.graphics’. Indica el código utilizado y el resultado obtenido. (1 punto)

In [ ]:
import spacy
import glob

nlp = spacy.load('en_core_web_sm')
directory_path = 'C:/Users/Del Cerro Recuero/.keras/datasets/20_newsgroup/comp.graphics/*'

#obtenemos  lista de archivos que coinciden con el patrón dado. 
sample_files = glob.glob(directory_path)[:15]

total_tokens = 0
num_files = 0

for file in sample_files:
    if os.path.isfile(file) and os.path.splitext(file)[1] == '':
#Abrimos el archivo, leemos su contenido, y pasamos al modelo de spaCy para obtener el documento procesado y actualizamos los contadores de tokens y archivos.
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            doc = nlp(text)
            total_tokens += len(doc)
            num_files += 1

#Calculamos el número promedio de tokens dividiendo el conteo total de tokens entre el número de archivos procesados.
average_tokens = total_tokens / num_files

print("Número promedio de tokens:", average_tokens)



### 2.	El código proporcionado lee los ficheros uno a uno y, antes de generar el catálogo de datos de entrenamiento y validación, descarta las 10 primeras líneas de cada fichero. ¿Cuál es el trozo de código en el que se realiza dicho descarte?, ¿por qué crees que se descartan dichas líneas?, ¿por qué 10 y no otro número? (1 punto)

Dentro del bluque 'for fname in fname' y es el siguiente: 
lines = content.split("\n")   
        lines = lines[10:]   
        content = "\n".join(lines)

El motivo por la que se descartan las primeras 10 líneas es porque, en este caso, se considera que esas líneas contienen información adicional o metadatos que no son relevantes para el análisis o encabezados del texto. Al eliminar estas líneas, se obtiene un conjunto de datos más limpio y enfocado en el contenido principal de los ficheros.

Al elegir más de 10 diez lineas se eliminaria información levelante para el objeto del analisis. 

### 3.	¿Qué se controla con el parámetro 'validation_split'?, ¿por qué se ha elegido ese valor?, ¿qué ocurre si lo modificas? (1 punto)

El parámetro 'validation_split' controla el tamaño del conjunto de datos que se utilizará como datos de validación durante el entrenamiento del modelo. Este parámetro indica el  porcentaje de los datos se separará para su uso en la validación, mientras que el resto se utilizará para el entrenamiento.

En el código, se ha indicado el valor de 'validation_split' en 0.2, lo que significa que el 20% de los datos se utilizará como conjunto de validación. Esta elección se basa en la buenas practicas empleadas en este modelos. 

Si se modifica el valor de 'validation_split' a un número diferente, cambiamos la proporción de datos que se utilizarán para la validación. Por ejemplo, si se establece en 0.3, se utilizará el 30% de los datos para la validación, dejando el 70% restante para el entrenamiento. Ajustar este valor puede afectar el rendimiento del modelo y su capacidad para generalizar a nuevos datos. Elegir un valor adecuado para 'validation_split' depende del tamaño del conjunto de datos, la cantidad de datos disponibles y las características específicas de los datos. 



### 4.	Imprime por pantalla un ejemplo (es decir, un elemento del array) de ‘train_samples’, ‘val_samples’, ‘train_labels’ y ‘val_labels’. A tenor de las etiquetas que se utilizan, ¿qué tarea crees que se está intentando entrenar? (1 punto)

In [ ]:
print("Ejemplo de train_samples:", train_samples[0])
print("Ejemplo de val_samples:", val_samples[0])
print("Ejemplo de train_labels:", train_labels[0])
print("Ejemplo de val_labels:", val_labels[0])


La tarea que se esté intentando entrenar es una tarea de clasificación de texto. Las etiquetas representan las categorías o clases a las que pertenecen los datos. Por ejemplo: computer graphic, alt.atheism, comp.sys.mac.hardware.





### 5.	Con 'output_sequence_length' se establece un tamaño fijo para la salida de Vectorizer. ¿Por qué se necesita un tamaño fijo, y por qué se ha elegido el valor ‘200’? (1 punto)

Es necesario establecer un tamaño fijo para poder procesar los datos en lotes de manera optima y para garantizar la consistencia en las dimensiones de entrada del modelo. 

¿Por qué se ha elegido el valor ‘200’?,el porque de un tamaño fijo es para busca limitar la longitud de las secuencias del texto a un tamaño manejable y representativo. 

### 6.	Indica cuál es la precisión del modelo en el conjunto de datos de entrenamiento y en el conjunto de datos de validación. ¿Qué interpretación puedes dar? Haz en este punto un análisis comparativo de los dos modelos ejecutados. (1.5 puntos)

En el modelo basado en Transformers, la precisión en el conjunto de datos de entrenamiento es de aproximadamente 93.91% después de 20 épocas, mientras que la precisión en el conjunto de datos de validación es de aproximadamente 79.74%. Esto significa que el modelo tiene un buen rendimiento en el conjunto de datos de entrenamiento, alcanzando una alta precisión, pero hay cierta discrepancia en el conjunto de datos de validación, lo que indica que el modelo puede estar sobre ajustando los datos de entrenamiento y no generalizando tan bien a nuevos datos.

Por otro lado, en el modelo de Red Neuronal Clásica, la precisión en el conjunto de datos de entrenamiento es de aproximadamente 96.58% después de 20 épocas, mientras que la precisión en el conjunto de datos de validación es de aproximadamente 71.57%. Aquí también se observa una discrepancia entre la precisión en el conjunto de entrenamiento y el de validación, lo que sugiere que el modelo puede estar sobre ajustando los datos de entrenamiento y no generalizando adecuadamente.

En comparación, el modelo basado en Transformers tiene una precisión ligeramente mayor tanto en el conjunto  de validación en comparación con el modelo de Red Neuronal Clásica. Sin embargo, ambos modelos presentan cierto grado de sobreajuste, lo que puede ser abordado mediante técnicas como regularización y aumento de datos para mejorar la generalización del modelo.

### 7.	En la parte final del código se hace un análisis cualitativo de la salida. Explica el funcionamiento de este análisis e interpreta los resultados. Haz también en este punto un análisis comparativo de los dos modelos ejecutados. (1 punto)

Red Neuronal Clásica: El primero comenta sobre tarjeta grafica y lo acierta indicando: comp.graphics
en cambio cuando habla de politica no lo acierta indicando: alt.atheismo y la tercera evaluacio tampoco lo acierta porque habla 
de religion y presenta comp.sys.mac.hardware. 

En el modelo basado en Transforml: El  primero si que lo acierta, el segundo parece que tambien lo acierta, pero la palabra "guns" no sabemos como la obtiene y la tercera si parece que esta hablando de religion, pero la palabra "alt" no sabemos el significado y ni el contexto. 


### 8.	Explica algunas de las limitaciones que puedes encontrar al modelo entrenado. (1.5 puntos)

A contoninuacion explicamos algunas limitaciones de este tipos de modelos.

    Ironia: Para un modelo puede ser complicado dectectar sarcasmo o ironia. En este caso son caracteristica del ser humano. 
    
    Idioma: Es necesario mantener actualizado el modelo  por que el idioma puede sufrir modidificaciones.
    
    Sesgo de datos: Si los datos de entrenamiento están sesgados hacia ciertas clases, el modelo puede tener dificultades para generalizar correctamente a clases menos representadas, lo que resulta en una precisión más baja para esas clases.

    Tamaño del conjunto de datos: El rendimiento del modelo puede verse afectado por el tamaño del conjunto de datos utilizado para entrenar. Si el conjunto de datos es pequeño, el modelo puede tener dificultades para aprender patrones generales y puede sufrir de sobreajuste, lo que significa que se adapta demasiado a los datos de entrenamiento y no generaliza bien a nuevos datos.

    Composición del texto: Dependiendo del tipo de texto que se está clasificando, pueden existir desafíos adicionales, como errores ortográficos, jerga o ambigüedad. Estos factores pueden dificultar la interpretación precisa del significado del texto por parte del modelo y afectar sus predicciones.

    Representación de palabras: El modelo utiliza embeddings para representar las palabras como vectores. La calidad de los embeddings y la representación de las palabras pueden afectar el rendimiento del modelo. Si las palabras clave relevantes no están bien capturadas en los vectores de embedding o si hay mucha variabilidad en las representaciones de palabras similares, el modelo puede tener dificultades para aprender las relaciones semánticas entre las palabras.

    Arquitectura del modelo: El modelo actual tiene una arquitectura relativamente simple con una capa de embedding, una capa de promedio global y dos capas densas. Esta arquitectura puede ser adecuada para algunos problemas, pero para problemas más complejos, puede ser necesario explorar arquitecturas más avanzadas, como redes neuronales recurrentes (RNN), redes neuronales convolucionales (CNN) o modelos basados en atención.

In [ ]:
import matplotlib.pyplot as plt

# Obtener la frecuencia de cada clase
class_frequencies = {}
for label in labels:
    if label in class_frequencies:
        class_frequencies[label] += 1
    else:
        class_frequencies[label] = 1

# Calcular el porcentaje de cada clase
total_instances = len(labels)
class_percentages = {label: freq / total_instances for label, freq in class_frequencies.items()}

# Visualizar la distribución de clases
labels = class_percentages.keys()
frequencies = class_percentages.values()

plt.bar(labels, frequencies)
plt.xlabel('Clases')
plt.ylabel('Porcentaje')
plt.title('Distribución de Clases')
plt.show()


### 9.	¿Qué sería necesario para que este modelo pueda interpretar textos en español? (1 punto)

Para permitir que el modelo interprete textos en español, es necesario seguir algunos pasos adicionales. Primero, se requiere un conjunto de datos etiquetados en español que contenga ejemplos de textos y sus respectivas etiquetas.

Además, sera necesario adaptar el preprocesamiento del texto para lidiar con las particularidades del español, como caracteres especiales, puntuación, acentos y otros aspectos específicos del idioma.

Para trabajar con textos en español, se necesitarán embeddings específicos del idioma. Estos embeddings son representaciones vectoriales de palabras que capturan su significado semántico. Puedes utilizar embeddings pre-entrenados en español, como FastText o Word2Vec, o entrenar tus propios embeddings en un corpus de texto en español.

Para evaluar el rendimiento del modelo, es necesario contar con etiquetas y datos de prueba en español. Esto implica traducir o recolectar datos etiquetados en español que representen las clases o categorías que deseas predecir.

Una vez que reunido todos los elementos que acabmos de destacar, se debera adaptar el código y realizar el entrenamiento del modelo utilizando el conjunto de datos en español, los embeddings y los modelos de lenguaje correspondientes.

# Evaluación

In [ ]:
from tensorflow.keras.models import load_model

string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloEmbeddingGloveTransformers(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]




from tensorflow.keras.models import load_model

# Carga el modelo previamente guardado
modeloEmbeddingGloveTransformers = load_model('ruta_del_modelo/modelo.h5')

# Continúa con el resto del código
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloEmbeddingGloveTransformers(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]


In [ ]:
probabilities = end_to_end_model.predict(
    [["politics and federal courts law that people understand with politician and elects congressman"]]
)

class_names[np.argmax(probabilities[0])]

In [ ]:
probabilities = end_to_end_model.predict(
    [["we are talking about religion"]]
)

class_names[np.argmax(probabilities[0])]